<a href="https://colab.research.google.com/github/mitesh1691/lp5/blob/main/HPC_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/afnan47/cuda.git

  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-bjikjxee
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-bjikjxee
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4289 sha256=a9a0a0e91e61005409b9d9e4bd5e8f7b53f63dfd6b6c9036ac42372026ac2a17
  Stored in directory: /tmp/pip-ephem-wheel-cache-0s4hxga3/wheels/aa/f3/44/e10c1d226ec561d971fcd4b0463f6bff08602afa928a3e7bc7
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include <iostream>
int main(){
  std::cout << "Hello World\n";
  return 0;
}

Hello World



# Assignment 1:

## DFS

In [ ]:
%%writefile dfs.cu

#include <iostream>
#include <vector>
#include <omp.h>

using namespace std;

// Graph class representing the adjacency list
class Graph {
    int V;  // Number of vertices
    vector<vector<int>> adj;  // Adjacency list

public:
    Graph(int V) : V(V), adj(V) {}

    // Add an edge to the graph
    void addEdge(int v, int w) {
        adj[v].push_back(w);
        adj[w].push_back(v);
    }


    // Parallel Depth-First Search
    void parallelDFS(int startVertex) {
        vector<bool> visited(V, false);
        double startTime = omp_get_wtime();
        parallelDFSUtil(startVertex, visited);
        double endTime = omp_get_wtime();
        cout << "\nExecution Time (DFS): " << endTime - startTime << " seconds" << endl;
    }

    // Parallel DFS utility function
    void parallelDFSUtil(int v, vector<bool>& visited) {
        visited[v] = true;
        cout << v << " ";

        #pragma omp parallel for
        for (int i = 0; i < adj[v].size(); i++) {
            int n = adj[v][i];
            if (!visited[n])
                parallelDFSUtil(n, visited);
        }
    }
};

int main() {
    int V, E;
    cout << "Enter the number of vertices: ";
    cin >> V;
    cout << "Enter the number of edges: ";
    cin >> E;

    // Create a graph
    Graph g(V);

    cout << "Enter edges (vertex pairs):\n";
    for (int i = 0; i < E; ++i) {
        int v, w;
        cin >> v >> w;
        if (v < 0 || v >= V || w < 0 || w >= V) {
            cout << "Invalid vertex index!" << endl;
            return 1; // Exit with error code
        }
        g.addEdge(v, w);
    }

    int startVertex;
    cout << "Enter the start vertex for DFS: ";
    cin >> startVertex;

    cout << "Depth-First Search (DFS): ";
    g.parallelDFS(startVertex);
    cout << endl;

    return 0;
}

Writing dfs.cu


In [ ]:
!nvcc -o dfs dfs.cu -Xcompiler -fopenmp
!./dfs

Enter the number of vertices: 5
Enter the number of edges: 6
Enter edges (vertex pairs):
0 1
0 2
1 3
1 2
2 4
3 4
Enter the start vertex for DFS: 0
Depth-First Search (DFS): 0 1 3 2 4 
Execution Time (DFS): 0.000164871 seconds



## BFS

In [ ]:
%%writefile bfs.cu

#include <iostream>
#include <vector>
#include <queue>
#include <omp.h>

using namespace std;

// Graph class representing the adjacency list
class Graph {
    int V;  // Number of vertices
    vector<vector<int>> adj;  // Adjacency list

public:
    Graph(int V) : V(V), adj(V) {}

    // Add an edge to the graph
    void addEdge(int v, int w) {
        adj[v].push_back(w);
        adj[w].push_back(v);
    }

    // Parallel Breadth-First Search
    void parallelBFS(int startVertex) {
        vector<bool> visited(V, false);
        queue<int> q;
        double startTime = omp_get_wtime();

        visited[startVertex] = true;
        q.push(startVertex);

        while (!q.empty()) {
            int v = q.front();
            q.pop();
            cout << v << " ";

            #pragma omp parallel for
            for (int i = 0; i < adj[v].size(); i++) {
                int n = adj[v][i];
                if (!visited[n]) {
                    visited[n] = true;
                    q.push(n);
                }
            }
        }

        double endTime = omp_get_wtime();
        cout << "\nExecution Time (BFS): " << endTime - startTime << " seconds" << endl;
    }
};

int main() {
    int V, E;
    cout << "Enter the number of vertices: ";
    cin >> V;
    cout << "Enter the number of edges: ";
    cin >> E;

    // Create a graph
    Graph g(V);

    cout << "Enter edges (vertex pairs):\n";
    for (int i = 0; i < E; ++i) {
        int v, w;
        cin >> v >> w;
        if (v < 0 || v >= V || w < 0 || w >= V) {
            cout << "Invalid vertex index!" << endl;
            return 1; // Exit with error code
        }
        g.addEdge(v, w);
    }

    int startVertex;
    cout << "Enter the start vertex for BFS: ";
    cin >> startVertex;

    cout << "Breadth-First Search (BFS): ";
    g.parallelBFS(startVertex);
    cout << endl;

    return 0;
}

Writing ass2.cu


In [ ]:
!nvcc -o bfs bfs.cu -Xcompiler -fopenmp
!./bfs

Enter the number of vertices: 5
Enter the number of edges: 6
Enter edges (vertex pairs):
0 1
0 2
1 3
1 2
2 4
3 4
Enter the start vertex for BFS: 0
Breadth-First Search (BFS): 0 1 2 3 4 
Execution Time (BFS): 0.00027463 seconds



# Assignment 2:


## Bubble Sort

In [ ]:
%%writefile bubble.cu


#include <iostream>
#include <cstdlib>
#include <ctime>
#include <omp.h>

using namespace std;

void bubble(int array[], int n) {
    for (int i = 0; i < n - 1; i++) {
        for (int j = 0; j < n - i - 1; j++) {
            if (array[j] > array[j + 1]) swap(array[j], array[j + 1]);
        }
    }
}

void pBubble(int array[], int n) {
    // Sort odd indexed numbers
    for (int i = 0; i < n; ++i) {
        #pragma omp for
        for (int j = 1; j < n; j += 2) {
            if (array[j] < array[j - 1]) {
                swap(array[j], array[j - 1]);
            }
        }

        // Synchronize
        #pragma omp barrier

        // Sort even indexed numbers
        #pragma omp for
        for (int j = 2; j < n; j += 2) {
            if (array[j] < array[j - 1]) {
                swap(array[j], array[j - 1]);
            }
        }
    }
}

void printArray(int arr[], int n) {
    for (int i = 0; i < n; i++) cout << arr[i] << " ";
    cout << "\n";
}

int main() {
    // Set up variables
    int n;
    cout << "Enter the size of the array: ";
    cin >> n;

    int arr[n];
    double start_time, end_time;

    // Seed the random number generator
    srand(time(0));

    // Fill the array with random numbers
    for (int i = 0; i < n; i++) arr[i] = rand() % 1000; // Generating random numbers between 0 to 999

    // Sequential time
    start_time = omp_get_wtime();
    bubble(arr, n);
    end_time = omp_get_wtime();
    cout << "Sequential Bubble Sort took: " << end_time - start_time << " seconds.\n";
    printArray(arr, n);

    // Parallel time
    start_time = omp_get_wtime();
    pBubble(arr, n);
    end_time = omp_get_wtime();
    cout << "Parallel Bubble Sort took: " << end_time - start_time << " seconds.\n";
    printArray(arr, n);

    return 0;
}

Writing bubble.cu


In [ ]:
!nvcc -o bubble bubble.cu -Xcompiler -fopenmp
!./bubble

Enter the size of the array: 15
Sequential Bubble Sort took: 2.238e-06 seconds.
112 138 170 272 333 377 421 480 480 557 582 591 619 715 808 
Parallel Bubble Sort took: 2.424e-06 seconds.
112 138 170 272 333 377 421 480 480 557 582 591 619 715 808 


## Merge Sort

In [ ]:
%%writefile merge.cu

#include <iostream>
#include <cstdlib>
#include <ctime>
#include <omp.h>

using namespace std;

void merge(int arr[], int low, int mid, int high) {
    // Create arrays of left and right partitions
    int n1 = mid - low + 1;
    int n2 = high - mid;

    int left[n1];
    int right[n2];

    // Copy all left elements
    for (int i = 0; i < n1; i++) left[i] = arr[low + i];

    // Copy all right elements
    for (int j = 0; j < n2; j++) right[j] = arr[mid + 1 + j];

    // Compare and place elements
    int i = 0, j = 0, k = low;

    while (i < n1 && j < n2) {
        if (left[i] <= right[j]) {
            arr[k] = left[i];
            i++;
        } else {
            arr[k] = right[j];
            j++;
        }
        k++;
    }

    // If any elements are left out
    while (i < n1) {
        arr[k] = left[i];
        i++;
        k++;
    }

    while (j < n2) {
        arr[k] = right[j];
        j++;
        k++;
    }
}

void parallelMergeSort(int arr[], int low, int high) {
    if (low < high) {
        int mid = (low + high) / 2;

        #pragma omp parallel sections
        {
            #pragma omp section
            {
                parallelMergeSort(arr, low, mid);
            }

            #pragma omp section
            {
                parallelMergeSort(arr, mid + 1, high);
            }
        }
        merge(arr, low, mid, high);
    }
}

void mergeSort(int arr[], int low, int high) {
    if (low < high) {
        int mid = (low + high) / 2;
        mergeSort(arr, low, mid);
        mergeSort(arr, mid + 1, high);
        merge(arr, low, mid, high);
    }
}

void printArray(int arr[], int n) {
    for (int i = 0; i < n; i++) cout << arr[i] << " ";
    cout << "\n";
}

int main() {
    int n;
    cout << "Enter the size of the array: ";
    cin >> n;

    int arr[n];
    double start_time, end_time;

    // Seed the random number generator
    srand(time(0));

    // Fill the array with random numbers
    for (int i = 0; i < n; i++) arr[i] = rand() % 1000; // Generating random numbers between 0 to 999

    // Measure Sequential Time
    start_time = omp_get_wtime();
    mergeSort(arr, 0, n - 1);
    end_time = omp_get_wtime();
    cout << "Time taken by sequential algorithm: " << end_time - start_time << " seconds\n";
    cout << "Sorted array (sequential): ";
    printArray(arr, n);

    // Measure Parallel time
    start_time = omp_get_wtime();
    parallelMergeSort(arr, 0, n - 1);
    end_time = omp_get_wtime();
    cout << "Time taken by parallel algorithm: " << end_time - start_time << " seconds\n";
    cout << "Sorted array (parallel): ";
    printArray(arr, n);

    return 0;
}

Writing merge.cu


In [ ]:
!nvcc -o merge merge.cu -Xcompiler -fopenmp
!./merge

Enter the size of the array: 15
Time taken by sequential algorithm: 2.258e-06 seconds
Sorted array (sequential): 32 122 130 140 147 223 240 388 428 428 677 772 825 929 974 
Time taken by parallel algorithm: 0.000158596 seconds
Sorted array (parallel): 32 122 130 140 147 223 240 388 428 428 677 772 825 929 974 


# Assignment 3:


## Parallel Reduction - Min, Max, Sum, AVG

In [ ]:
%%writefile ass3.cu

#include<iostream>
#include<omp.h>
#include<cstdlib>
#include<ctime>

using namespace std;

int minval(int arr[], int n){
    int minval = arr[0];
    #pragma omp parallel for reduction(min : minval)
    for(int i = 0; i < n; i++){
        if(arr[i] < minval) minval = arr[i];
    }
    return minval;
}

int maxval(int arr[], int n){
    int maxval = arr[0];
    #pragma omp parallel for reduction(max : maxval)
    for(int i = 0; i < n; i++){
        if(arr[i] > maxval) maxval = arr[i];
    }
    return maxval;
}

int sum(int arr[], int n){
    int sum = 0;
    #pragma omp parallel for reduction(+ : sum)
    for(int i = 0; i < n; i++){
        sum += arr[i];
    }
    return sum;
}

int average(int arr[], int n){
    return (double)sum(arr, n) / n;
}

int main(){
    int n;
    cout << "Enter the size of the array: ";
    cin >> n;

    int arr[n];

    // Seed the random number generator
    srand(time(0));

    // Fill the array with random numbers
    cout << "Generated array: ";
    for(int i = 0; i < n; i++) {
        arr[i] = rand() % 100; // Generating random numbers between 0 to 99
        cout << arr[i] << " ";
    }
    cout << endl;

    cout << "The minimum value is: " << minval(arr, n) << '\n';
    cout << "The maximum value is: " << maxval(arr, n) << '\n';
    cout << "The summation is: " << sum(arr, n) << '\n';
    cout << "The average is: " << average(arr, n) << '\n';
    return 0;
}

Writing ass1.cu


In [ ]:
!nvcc -o ass3 ass3.cu -Xcompiler -fopenmp
!./ass3

Enter the size of the array: 10
Generated array: 65 33 51 40 36 45 60 37 4 97 
The minimum value is: 4
The maximum value is: 97
The summation is: 468
The average is: 46


# Assignment 4:


## Vector Addition

In [ ]:
%%writefile vectorAdd.cu

#include <iostream>
#include <cstdlib>
#include <ctime>

using namespace std;

__global__ void add(int* A, int* B, int* C, int size) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    if (tid < size) {
        C[tid] = A[tid] + B[tid];
    }
}

void initializeRandom(int* vector, int size, int seed) {
    srand(seed);
    for (int i = 0; i < size; i++) {
        vector[i] = rand() % 100; // Generating random numbers between 0 to 99
    }
}

void print(int* vector, int size) {
    for (int i = 0; i < size; i++) {
        cout << vector[i] << " ";
    }
    cout << endl;
}

int main() {
    int N;
    cout << "Enter the size of the vectors: ";
    cin >> N;

    int* A, * B, * C;

    int vectorSize = N;
    size_t vectorBytes = vectorSize * sizeof(int);

    A = new int[vectorSize];
    B = new int[vectorSize];
    C = new int[vectorSize];

    // Use the new function to initialize vectors with different random elements
    initializeRandom(A, vectorSize, 1234); // Use different seed for vector A
    initializeRandom(B, vectorSize, 5678); // Use different seed for vector B

    cout << "Vector A: ";
    print(A, N);
    cout << "Vector B: ";
    print(B, N);

    int* X, * Y, * Z;
    cudaMalloc(&X, vectorBytes);
    cudaMalloc(&Y, vectorBytes);
    cudaMalloc(&Z, vectorBytes);

    cudaMemcpy(X, A, vectorBytes, cudaMemcpyHostToDevice);
    cudaMemcpy(Y, B, vectorBytes, cudaMemcpyHostToDevice);

    int threadsPerBlock = 256;
    int blocksPerGrid = (N + threadsPerBlock - 1) / threadsPerBlock;

    add<<<blocksPerGrid, threadsPerBlock>>>(X, Y, Z, N);

    cudaMemcpy(C, Z, vectorBytes, cudaMemcpyDeviceToHost);

    cout << "Addition: ";
    print(C, N);

    delete[] A;
    delete[] B;
    delete[] C;

    cudaFree(X);
    cudaFree(Y);
    cudaFree(Z);

    return 0;
}


Overwriting vectorAdd.cu


In [ ]:
!nvcc -o vectorAdd vectorAdd.cu -Xcompiler -fopenmp
!./vectorAdd

Enter the size of the vectors: 20
Vector A: 14 39 55 67 60 41 55 98 72 53 75 41 66 33 68 71 58 55 64 1 
Vector B: 56 79 16 91 27 21 33 15 54 85 87 76 41 81 30 92 99 14 13 62 
Addition: 70 118 71 158 87 62 88 113 126 138 162 117 107 114 98 163 157 69 77 63 


## Matrix Multiplication

In [ ]:
%%writefile matMul.cu


#include <iostream>
using namespace std;

// CUDA code to multiply matrices
__global__ void multiply(int* A, int* B, int* C, int size) {
    // Uses thread indices and block indices to compute each element
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < size && col < size) {
        int sum = 0;
        for (int i = 0; i < size; i++) {
            sum += A[row * size + i] * B[i * size + col];
        }
        C[row * size + col] = sum;
    }
}

void initialize(int* matrix, int size) {
    for (int i = 0; i < size * size; i++) {
        matrix[i] = rand() % 10;
    }
}

void print(int* matrix, int size) {
    for (int row = 0; row < size; row++) {
        for (int col = 0; col < size; col++) {
            cout << matrix[row * size + col] << " ";
        }
        cout << '\n';
    }
    cout << '\n';
}

int main() {
    int N;
    cout << "Enter the size of the matrices: ";
    cin >> N;

    int* A, * B, * C;

    int matrixSize = N * N;
    size_t matrixBytes = matrixSize * sizeof(int);

    A = new int[matrixSize];
    B = new int[matrixSize];
    C = new int[matrixSize];

    initialize(A, N);
    initialize(B, N);
    cout << "Matrix A: \n";
    print(A, N);

    cout << "Matrix B: \n";
    print(B, N);

    int* X, * Y, * Z;
    // Allocate space
    cudaMalloc(&X, matrixBytes);
    cudaMalloc(&Y, matrixBytes);
    cudaMalloc(&Z, matrixBytes);

    // Copy values from A to X
    cudaMemcpy(X, A, matrixBytes, cudaMemcpyHostToDevice);

    // Copy values from A to X and B to Y
    cudaMemcpy(Y, B, matrixBytes, cudaMemcpyHostToDevice);

    // Threads per CTA dimension
    int THREADS = 2;

    // Blocks per grid dimension (assumes THREADS divides N evenly)
    int BLOCKS = (N + THREADS - 1) / THREADS;

    // Use dim3 structs for block  and grid dimensions
    dim3 threads(THREADS, THREADS);
    dim3 blocks(BLOCKS, BLOCKS);

    // Launch kernel
    multiply<<<blocks, threads>>>(X, Y, Z, N);

    cudaMemcpy(C, Z, matrixBytes, cudaMemcpyDeviceToHost);
    cout << "Multiplication of matrices A and B: \n";
    print(C, N);

    delete[] A;
    delete[] B;
    delete[] C;

    cudaFree(X);
    cudaFree(Y);
    cudaFree(Z);

    return 0;
}



Writing matMul.cu


In [ ]:
!nvcc -o matMul matMul.cu -Xcompiler -fopenmp
!./matMul

Enter the size of the matrices: 10
Matrix A: 
3 6 7 5 3 5 6 2 9 1 
2 7 0 9 3 6 0 6 2 6 
1 8 7 9 2 0 2 3 7 5 
9 2 2 8 9 7 3 6 1 2 
9 3 1 9 4 7 8 4 5 0 
3 6 1 0 6 3 2 0 6 1 
5 5 4 7 6 5 6 9 3 7 
4 5 2 5 4 7 4 4 3 0 
7 8 6 8 8 4 3 1 4 9 
2 0 6 8 9 2 6 6 4 9 

Matrix B: 
5 0 4 8 7 1 7 2 7 2 
2 6 1 0 6 1 5 9 4 9 
0 9 1 7 7 1 1 5 9 7 
7 6 7 3 6 5 6 3 9 4 
8 1 2 9 3 9 0 8 8 5 
0 9 6 3 8 5 6 1 1 5 
9 8 4 8 1 0 3 0 4 4 
4 4 7 6 3 1 7 5 9 6 
2 1 7 8 5 7 4 1 8 5 
9 7 5 3 8 8 3 1 8 9 

Multiplication of matrices A and B: 
175 249 202 265 250 166 189 159 304 259 
193 221 206 158 243 179 207 162 271 248 
189 237 189 207 250 171 178 179 330 275 
248 213 227 286 262 198 226 182 330 233 
244 233 248 291 255 168 247 141 312 224 
114 107 104 161 146 129 103 123 169 159 
291 295 263 305 297 214 250 204 392 322 
155 196 173 198 201 132 179 140 226 197 
291 289 230 295 341 254 226 226 400 339 
305 268 239 312 263 245 183 175 396 294 

